In [20]:
#ingest data
import pandas as pd
import pickle
from sklearn.linear_model import Lasso



#For Evaluating the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Algorithm used - LinearRegression
from sklearn.linear_model import LinearRegression

#To convert million , billion string into float
import re

#For plotting
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer

import seaborn as sns
import matplotlib.pyplot as plt

import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("youtube-prediction-experiment")

<Experiment: artifact_location='/workspaces/MLOps_Zoomcamp_Project_YoutubePrediction/02-Experiment_Tracking/mlruns/1', creation_time=1720106073958, experiment_id='1', last_update_time=1720106073958, lifecycle_stage='active', name='youtube-prediction-experiment', tags={}>

In [21]:
# Load the data
def read_dataframe(filename):
    data = pd.read_csv(filename)

    # Drop rows with NaN values
    data.dropna(inplace=True)
    
    data['VIEWS'] = clean_numeric_column(data['VIEWS'])
    data['TOTAL_NUMBER_OF_VIDEOS'] = clean_numeric_column(data['TOTAL_NUMBER_OF_VIDEOS'])
    data['SUBSCRIBERS'] = clean_numeric_column(data['SUBSCRIBERS'])
    
    print(data)
    
    
    return data


    
def clean_numeric_column(column):
    cleaned_column = []
    for value in column:
        if isinstance(value, str):
            if 'K' in value:
                cleaned_column.append(float(re.sub(r'[^0-9.]', '', value)) * 1000)
            elif 'M' in value:
                cleaned_column.append(float(re.sub(r'[^0-9.]', '', value)) * 1000000)
            elif 'B' in value:
                cleaned_column.append(float(re.sub(r'[^0-9.]', '', value)) * 1000000000)
            else:
                cleaned_column.append(float(re.sub(r'[^0-9.]', '', value)))
        else:
            cleaned_column.append(value)
    return cleaned_column

In [22]:
df_train = read_dataframe('/workspaces/MLOps_Zoomcamp_Project_YoutubePrediction/dataset/youtube_test.csv')
df_val = read_dataframe('/workspaces/MLOps_Zoomcamp_Project_YoutubePrediction/dataset/youtube_train.csv')

     RANK             NAME_OF_CHANNEL  TOTAL_NUMBER_OF_VIDEOS  SUBSCRIBERS  \
0      #1                     MrBeast                   799.0  274000000.0   
1      #2                    T-Series                 21120.0  267000000.0   
2      #3  Cocomelon - Nursery Rhymes                  1180.0  176000000.0   
3      #4                   SET India                138970.0  173000000.0   
4      #5           ✿ Kids Diana Show                  1220.0  123000000.0   
..    ...                         ...                     ...          ...   
491  #492       SUPER SLICK SLIME SAM                  1180.0   20600000.0   
492  #493              Clash of Clans                   868.0   20600000.0   
494  #495                 What's Idea                   205.0   20600000.0   
495  #496        Smile Family Spanish                   563.0   20500000.0   
496  #497        Marvel Entertainment                  8970.0   20500000.0   

            VIEWS                  CATEGORY  
0    5.098000e+10

In [23]:
print(len(df_train))
print(len(df_val))

485
485


In [24]:
# Define features and target
numeric_features = ['VIEWS', 'TOTAL_NUMBER_OF_VIDEOS']
categorical_features = ['CATEGORY']

# Prepare data for training
train_dicts = df_train[categorical_features + numeric_features].to_dict(orient='records')

# Prepare data for training
val_dicts = df_val[categorical_features + numeric_features].to_dict(orient='records')



# Initialize DictVectorizer
dv = DictVectorizer()


X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

print("X_train shape:", X_train.shape)

# Extract target variable
y_train = df_train['SUBSCRIBERS'].values
y_val = df_val['SUBSCRIBERS'].values
print("y_train shape:", y_train.shape)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

X_train shape: (485, 34)
y_train shape: (485,)


/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


14801134.038355662

In [25]:
# Predict the number of subscribers for each channel in the dataset
df_val['Predicted Subscribers'] = lr.predict(X_val)

# Save the predictions to a new CSV file
df_val.to_csv('/workspaces/MLOps_Zoomcamp_Project_YoutubePrediction/resultset/Result_youtube_channels_with_predictions.csv', index=False)
print("Predictions saved to 'youtube_channels_with_predictions.csv'")

Predictions saved to 'youtube_channels_with_predictions.csv'


In [26]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Anamika")

    mlflow.log_param("train-data-path", "/workspaces/mlops_zoomcamp_project/dataset/youtube_train.csv")
    mlflow.log_param("valid-data-path", "/workspaces/mlops_zoomcamp_project/dataset/youtube_test.csv")
    

    alpha = 0.2
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="/workspaces/MLOps_Zoomcamp_Project_YoutubePrediction/02-Experiment_Tracking/models", artifact_path="models_pickle")

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.033255250603581e+16, tolerance: 26926150350515.47
  model = cd_fast.sparse_enet_coordinate_descent(
/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [27]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [28]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [29]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=10,
            evals=[(valid, 'validation')],
            early_stopping_rounds=2
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [30]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

[0]	validation-rmse:22583269.44678                    
[1]	validation-rmse:21670441.98671                    
[2]	validation-rmse:20816503.86579                    
[3]	validation-rmse:20021491.35213                    
[4]	validation-rmse:19277488.75949                    
[5]	validation-rmse:18580266.30995                    
[6]	validation-rmse:17957233.74589                    
[7]	validation-rmse:17376030.54924                    
[8]	validation-rmse:16816065.99302                    
[9]	validation-rmse:16297621.89738                    
 10%|█         | 1/10 [00:00<00:02,  3.33trial/s, best loss: 16297622.00141467]

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:17:49] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:19678995.70645                                             
[1]	validation-rmse:17938257.23294                                             
[2]	validation-rmse:17199096.49183                                             
[3]	validation-rmse:16870050.31967                                             
[4]	validation-rmse:16329171.02270                                             
[5]	validation-rmse:16152282.54916                                             
[6]	validation-rmse:15917857.46693                                             
[7]	validation-rmse:15813758.78671                                             
[8]	validation-rmse:15534753.33917                                             
[9]	validation-rmse:15417243.01646                                             
 20%|██        | 2/10 [00:00<00:02,  3.71trial/s, best loss: 15417243.024033654]

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:17:50] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:16849837.24920                                              
[1]	validation-rmse:13785089.12193                                              
[2]	validation-rmse:12358098.32478                                              
[3]	validation-rmse:11224173.03821                                              
[4]	validation-rmse:10176278.98660                                              
[5]	validation-rmse:9565977.18208                                               
[6]	validation-rmse:8906429.97742                                               
[7]	validation-rmse:7966850.28821                                               
[8]	validation-rmse:7650846.47047                                               
[9]	validation-rmse:7263744.07296                                               
 30%|███       | 3/10 [00:00<00:01,  3.55trial/s, best loss: 7263744.061260423] 

...

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:17:51] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:20912261.79270                                             
[1]	validation-rmse:18560356.21767                                             
[2]	validation-rmse:16472995.85715                                             
[3]	validation-rmse:14620425.81990                                             
[4]	validation-rmse:12976232.12858                                             
[5]	validation-rmse:11516984.84136                                             
[6]	validation-rmse:10221875.29119                                             
[7]	validation-rmse:9072428.18126                                              
[8]	validation-rmse:8052267.66187                                              
[9]	validation-rmse:7146854.28623                                              
 90%|█████████ | 9/10 [00:02<00:00,  3.36trial/s, best loss: 3217344.315256135]

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:17:52] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:19247631.84008                                             
[1]	validation-rmse:16889550.08118                                             
[2]	validation-rmse:15265848.50758                                             
[3]	validation-rmse:14248015.97548                                             
[4]	validation-rmse:13657937.61243                                             
[5]	validation-rmse:13121690.62632                                             
[6]	validation-rmse:12757686.29972                                             
[7]	validation-rmse:12486065.47131                                             
[8]	validation-rmse:12263574.86082                                             
[9]	validation-rmse:12027510.86892                                             
100%|██████████| 10/10 [00:02<00:00,  3.44trial/s, best loss: 3217344.315256135]


/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:17:52] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

